In [2]:
import pandas as pd
import pickle
import numpy as np

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
def create_similarity():
    data = pd.read_csv('problem-detaset.csv')
    cv = CountVectorizer()
    count_matrix = cv.fit_transform(data['tags']).toarray()
    similarity = cosine_similarity(count_matrix)
    return data,similarity

In [16]:
def recommended(tag,id,with_url=None,lenght=10):
    tag = tag.lower()
    data, similarity = create_similarity()
    if tag not in data['tags'].unique():
        return('Tag not found')
    else:
        try:
            i = data[(data['tags']==tag)].index[0]
            lst = list(enumerate(similarity[i]))
            lst = sorted(lst, key = lambda x:x[1] ,reverse=True)
            l = []
            for i in range((lenght)+1):
                a = lst[i][0]
                temp =[]
                if data['id'][a]==id:
                    lenght+=1
                    continue
                if with_url is None:
                    temp.append(data['tags'][a])
                    temp.append(data['id'][a])
                else:
                    temp.append(data['tags'][a])
                    url = 'http://arena.siesgst.ac.in/contest/'+str(data['contestCode'][a])+'/problem/'+str(data['code'][a])
                    temp.append(url)
                l.append(temp)
            return l
        except Exception as e:
            print(e)
            return('No data found')

In [22]:
r = recommended('cakewalk','5c39f0543646190022847c61',with_url=True)
print(r)
print("\n\n")
r = recommended('cakewalk','5c39ed573646190022847c59')
print((r),len(r))
print("\n\n")
r = recommended('cakewalk','5c39ed573646190022847c59',lenght=6)
print((r),len(r))


[['cakewalk', 'http://arena.siesgst.ac.in/contest/UNIVERSE/problem/UNI02'], ['cakewalk', 'http://arena.siesgst.ac.in/contest/UNIVERSE/problem/UNI04'], ['cakewalk', 'http://arena.siesgst.ac.in/contest/SRM22/problem/SRM22A'], ['cakewalk  math', 'http://arena.siesgst.ac.in/contest/SRM10/problem/SRM10A'], ['adhoc  cakewalk', 'http://arena.siesgst.ac.in/contest/SRM12/problem/SRM12A'], ['math', 'http://arena.siesgst.ac.in/contest/UNIVERSE/problem/UNI01'], ['math', 'http://arena.siesgst.ac.in/contest/UNIVERSE/problem/UNI05'], ['math  strings', 'http://arena.siesgst.ac.in/contest/UNIVERSE/problem/UNI06'], ['adhoc  strings', 'http://arena.siesgst.ac.in/contest/AUG18/problem/AUG18A'], ['adhoc  math', 'http://arena.siesgst.ac.in/contest/AUG18/problem/AUG18B']]



[['cakewalk', '5c39eca03646190022847c54'], ['cakewalk', '5c39f0543646190022847c61'], ['cakewalk', '60fa79a49e8f310022c7d341'], ['cakewalk  math', '5d46f3ce86031f00223f4ce7'], ['adhoc  cakewalk', '5d7a599fb112e300220a44d0'], ['math', '5b5

In [20]:
def prevoius_solve_tag(username):
    pass

In [21]:
data = pd.read_csv('problem-detaset.csv')
data[data['id']=='5c39f0543646190022847c61']

,id,name,code,points,contestCode,tags
2,5c39f0543646190022847c61,Vowel Shovel,UNI03,0,UNIVERSE,cakewalk
